 # WK 1/Phase 1 - Ingestion and Cleaning
 ## Variable Selection

In [1]:
import pandas as pd
import os
from sys import platform
import matplotlib.pyplot as plt
import datetime
import numpy as np
import pickle
import seaborn

In [2]:
def is_integer(x):
    '''
    This function returns True if x is an integer, and False otherwise
    '''
    try:
        return (int(x) == float(x))
    except:
        return False

 ## Parameters

In [3]:
#dir_data = "C:\\Users\\rodol\\OneDrive\\Documents\\Training\\Carnegie_Mellon_University\\Courses\\Mini_7\\Business_Value_Analytics\\General\\WK1\\Data"
dir_data = "C:\\Users\\ly266e\\Documents\\Training\\CMU\\Master\\Fall 2023 Mini 7\\Business_Analytics\\HW\\HW5\\Update\\Data"

 ## Step 1 - Ingestion
 Ingest the data files from both sets, perform consistency checks, and prepare one single file for each set

In [4]:
from typing import Dict, final
def ingest_files(directory: str) -> Dict:
    '''
    This function will ingest every file in the specified directory
    into a pandas dataframe. It will return a dictionary containing
    these dataframes, keyed by the file name.
    
    We assume the directory contains files directly downloaded from
    Lending Club, and *only* those files. Thus, we assume the files are zipped
    (pd.read_csv can read zipped files) and we assume the first line
    in each file needs to be skipped. 
    
    Note that this function will read and ingest more than one file and is
    convenient if you want to ingest data for more than one year at a time.
    
    Note that each file will be read *without* formatting
    '''
    
    # If the directory has no trailing slash, add one
    if directory[-1] != "/":
        directory = directory + "/"
    
    all_files = os.listdir(directory)
    output = {}
    
    print("Directory " + directory + " has " + str(len(all_files)) + " files:")
    for i in all_files:
        print("    Reading file " + i)
        output[i] = pd.read_csv(directory + i, dtype = str, skiprows = 1)
        
        # Some of the files have "summary" lines that, for example
        # read "Total number of loans number in Policy 1: ....."
        # To remove those lines, find any lines with non-integer IDs
        # and remove them
        invalid_rows = (output[i].id.apply( lambda x : is_integer(x) == False ))
        if invalid_rows.sum() > 0:
            print("        Found " + str(invalid_rows.sum()) + " invalid rows which were removed")
            output[i] = output[i][invalid_rows == False]
    
    return output

In [5]:
# Ingest the set of files we downloaded 
files_data = ingest_files(dir_data)

Directory C:\Users\ly266e\Documents\Training\CMU\Master\Fall 2023 Mini 7\Business_Analytics\HW\HW5\Update\Data/ has 1 files:
    Reading file LoanStats3c.csv
        Found 4 invalid rows which were removed


In [6]:
files_data.keys()

dict_keys(['LoanStats3c.csv'])

In [7]:
data_now = pd.concat(files_data.values()).reset_index(drop = True)
columns = list(data_now.columns)

In [8]:
print(f"The number of columns is: {len(columns)}")

The number of columns is: 144


In [9]:
print("Starting with " + str(len(data_now)) + " rows")

Starting with 235629 rows


 ## Step 2 - Choose Columns and Prepare Dataset

In [10]:
# Identify the columns we'll be keeping from the dataset
cols_to_pick = ['id','loan_amnt','funded_amnt','term','int_rate'
                ,'grade','emp_length', 'home_ownership',
                'annual_inc','verification_status','issue_d',
                'loan_status','purpose','dti', 'delinq_2yrs','open_acc','pub_rec',
                'revol_bal','revol_util', 'total_pymnt', 'recoveries',
                'inq_last_6mths', 'pct_tl_nvr_dlq', 'last_pymnt_d', 'earliest_cr_line']

# Identify the type of each of these column
float_cols = ['loan_amnt', 'funded_amnt', 'annual_inc',
              'dti', 'revol_bal', 'delinq_2yrs', 'open_acc', 'pub_rec',
              'total_pymnt', 'recoveries', 'inq_last_6mths']

cat_cols = ['term', 'grade', 'emp_length', 'home_ownership',
                    'verification_status', 'loan_status', 'purpose']

perc_cols = ['int_rate', 'revol_util', 'pct_tl_nvr_dlq']

date_cols = ['issue_d', 'last_pymnt_d', 'earliest_cr_line']

# Ensure that we have types for every column
assert set(cols_to_pick) - set(float_cols) - set(cat_cols) - set(perc_cols) - set(date_cols) == set(["id"])

In [11]:
# Keep only the columns of interest
final_data = data_now[cols_to_pick].copy()

In [12]:
print("Starting with " + str(len(final_data)) + " rows")

Starting with 235629 rows


 # To do (A)

 Choose 3 to 5 variables and add them to the list of variables below

 You should consult the data description (excel) file you downloaded to understand the definition of various available columns

 TIP: If you added new variables, be sure to clean them as we just did for the default variables.

 You will have to add them to the group of the right type of variables (e.g. percentage, date, categorical)


 ### Typecast the columns

In [13]:
for i in float_cols:
    final_data[i] = final_data[i].astype(float)

In [14]:
#####################################
def clean_perc(x):
    if pd.isnull(x):
        return np.nan
    
    # If x is already a float (or int), return it as is
    if isinstance(x, (float, int)):
        return float(x)
    
    x = x.rstrip()
    if x.endswith('%'):
        x = x[:-1]
        
    if x == '':
        return np.nan
    else:
        return float(x)
    
for i in perc_cols:
    final_data[i] = final_data[i].apply( clean_perc )

In [15]:
#######################################
def clean_date(x):
    if pd.isnull(x):
        return None
    else:
        return datetime.datetime.strptime(x, "%b-%y").date()

# Assuming final_data is your DataFrame and date_cols contains the date columns
for i in date_cols:
    final_data[i] = final_data[i].apply(clean_date)

In [16]:
#########################################
for i in cat_cols:
    final_data.loc[final_data[i].isnull(), i] = None

In [17]:
print("Number of Rows so far " + str(len(final_data)))

Number of Rows so far 235629


 ## Step 4 - Handle outliers

In [18]:
# There are quite a few outliers, but the two most obvious
# ones to remove are in annual_inc, revol_bal Remove these.
n_rows = len(final_data)
final_data = final_data[final_data.annual_inc < 1000000]
final_data = final_data[final_data.revol_bal < 400000]
final_data = final_data[final_data.dti < 200]
print("Removed " + str(n_rows - len(final_data)) + " rows")

Removed 78 rows


In [19]:
# Remove all loans that are too recent to have been paid off or defaulted
n_rows = len(final_data)
final_data = final_data[final_data.loan_status.isin(['Fully Paid','Charged Off','Default'])]
print("Removed " + str(n_rows - len(final_data)) + " rows")

Removed 0 rows


In [20]:
# Only include loans issued since 2009
n_rows = len(final_data)
final_data = final_data[final_data.issue_d >= datetime.date(2009, 1, 1)]
print("Removed " + str(n_rows - len(final_data)) + " rows")

Removed 42644 rows


### Outlier detection and Data Cleaning

In [21]:
def flag_and_limit_outliers(df, cols, z_threshold=3):
    from scipy import stats
    
    """
    Flag outliers in specified columns using Z-score method and return the upper limit.
    
    Parameters:
        df (DataFrame): Input DataFrame
        cols (list): List of columns to check for outliers
        z_threshold (float): Z-score threshold for flagging outliers. Defaults to 3.
        
    Returns:
        DataFrame: DataFrame with additional columns to flag outliers
        dict: Dictionary with upper limits for each column
    """
    df_outliers = df.copy()
    upper_limits = {}
    
    for col in cols:
        col_mean = df_outliers[col].mean()
        col_std = df_outliers[col].std()
        
        # Calculate upper limit for each column
        upper_limit = col_mean + (z_threshold * col_std)
        upper_limits[col] = upper_limit
        
        # Flag outliers in DataFrame
        df_outliers[f"{col}_outlier"] = (df_outliers[col] > upper_limit).astype(int)
        
    return df_outliers, upper_limits

In [22]:
columns_to_check = ['annual_inc', 'inq_last_6mths', 'total_pymnt', 'open_acc']

# Flag outliers and get upper limits
df_outliers_flagged, upper_limits = flag_and_limit_outliers(final_data, columns_to_check)

In [23]:
outliers_columns = ['annual_inc', 'inq_last_6mths', 'total_pymnt', 'open_acc']
upper_limits_filtered = {k: upper_limits[k] for k in outliers_columns if k in upper_limits}

In [24]:
# Remove outliers based on upper limits
n_rows = len(final_data)
for col, upper_limit in upper_limits_filtered.items():
    final_data = final_data[final_data[col] <= upper_limit]
print("Removed " + str(n_rows - len(final_data)) + " rows")

Removed 11169 rows


 ### Drop null values

In [25]:
# Deal with null values. We allow categorical variables to be null
# OTHER than grade, which is a particularly important categorical.
# All non-categorical variables must be non-null, and we drop rows that do not meet this requirement
required_cols = set(cols_to_pick) - set(cat_cols) - set(["id"])
required_cols.add("grade")

n_rows = len(final_data)
final_data.dropna(subset = required_cols ,inplace=True)
print("Removed " + str(n_rows - len(final_data)) + " rows")

Removed 193 rows


In [26]:
# Some of the columns selected will not be used directly in the model, but will be used to generate other features.
# Create variables specifying the features that will be used

# All categorical columns other than "loan_status" will be used as discrete features
discrete_features = list(set(cat_cols) - set(["loan_status"]))

# All numeric columns will be used as continuous features
continuous_features = list(float_cols + perc_cols)

In [27]:
final_data.shape

(181545, 25)

 ## Step 5 - Save a Pickle

In [28]:
# Remove the total_pymnt from the list of continuous features; this variable is highly predictive of the outcome but is not known at the time the loan is issued
continuous_features = [i for i in continuous_features if i not in ["total_pymnt", "recoveries"]]

In [29]:
# Define the output path for the pickle
pickle_file = "/".join(['.', "PickleData", "clean_data_feature.pickle"])
os.makedirs(os.path.dirname(pickle_file), exist_ok=True)
pickle.dump( [final_data, discrete_features, continuous_features], open(pickle_file, "wb") )

In [30]:
Difference = len(data_now) - len(final_data)
Percent_delta = Difference*100/len(data_now)
print('The Drop of data is ' + str(round(Percent_delta)) + '%')

The Drop of data is 23%


In [31]:
final_data.to_csv('C:\\Users\\ly266e\\Documents\\Training\\CMU\\Master\\Fall 2023 Mini 7\\Business_Analytics\\HW\\HW5\\Update\\Data\\clean_file_feature.csv', index=False)

 ## Read from Pickle if Saved
 Read data from saved pickle files

In [32]:
### Read the data and features from the pickle
final_data, discrete_features, continuous_features = pickle.load( open( "./PickleData/clean_data_feature.pickle", "rb" ) )

In [33]:
final_data.shape

(181545, 25)